# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/7849/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1022,Tim,0.42357390096531766,0.9970579258385417
2000-01-01 00:00:01,1069,Edith,-0.5386308016218473,0.5593376263936578
2000-01-01 00:00:02,971,Hannah,-0.01700926209609288,-0.8767734135912029
2000-01-01 00:00:03,938,Zelda,0.387757434079568,0.6449439237571684
2000-01-01 00:00:04,1039,Xavier,0.4858653807839419,0.7360449435138825
2000-01-01 00:00:05,948,Charlie,-0.7352633653487948,0.7599216642511171
2000-01-01 00:00:06,999,Hannah,-0.759517236102841,0.10875176703744804
2000-01-01 00:00:07,979,Zelda,-0.12840174307220242,-0.763216683375108
2000-01-01 00:00:08,1007,Oliver,-0.6089626649289805,0.2663327827913713


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1020,Ingrid,-0.619223986620554,0.6484650670427192
2000-01-30 00:00:01,997,Ray,-0.13842954824331,0.43957466333672457
2000-01-30 00:00:02,922,Kevin,0.8762453656890132,-0.9322749314505605
2000-01-30 00:00:03,1065,Zelda,-0.2578449087825172,0.8576301787705629
2000-01-30 00:00:04,1045,Ray,-0.9826472097113275,0.2842710671472539
2000-01-30 00:00:05,1037,Zelda,0.5968829539754306,0.5367026124300778
2000-01-30 00:00:06,990,George,0.9270398984905119,-0.8849441377229195
2000-01-30 00:00:07,964,Laura,0.15029307845098083,-0.6227665531235349
2000-01-30 00:00:08,1030,Bob,0.32863560308517803,0.39139832923330187


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1022,Tim,0.423574,0.997058
1,2000-01-01 00:00:01,1069,Edith,-0.538631,0.559338
2,2000-01-01 00:00:02,971,Hannah,-0.017009,-0.876773
3,2000-01-01 00:00:03,938,Zelda,0.387757,0.644944
4,2000-01-01 00:00:04,1039,Xavier,0.485865,0.736045


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1022,Tim,0.423574,0.997058
1,2000-01-01 00:00:01,1069,Edith,-0.538631,0.559338
2,2000-01-01 00:00:02,971,Hannah,-0.017009,-0.876773
3,2000-01-01 00:00:03,938,Zelda,0.387757,0.644944
4,2000-01-01 00:00:04,1039,Xavier,0.485865,0.736045


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.33 s, sys: 431 ms, total: 3.76 s
Wall time: 3.21 s


name
Alice      -1.565881e-03
Bob        -3.429515e-03
Charlie    -1.754424e-03
Dan        -5.897803e-04
Edith       1.238215e-03
Frank       1.084501e-04
George     -1.650859e-03
Hannah     -4.881836e-04
Ingrid     -9.907006e-04
Jerry       2.702808e-04
Kevin      -1.816574e-03
Laura      -1.765015e-03
Michael    -1.616450e-03
Norbert     2.764356e-03
Oliver     -2.794524e-03
Patricia    2.561128e-03
Quinn       2.019727e-03
Ray         1.039835e-03
Sarah      -1.200362e-03
Tim        -1.493787e-04
Ursula      2.147744e-03
Victor      7.983312e-07
Wendy       4.596371e-03
Xavier      3.172719e-03
Yvonne     -2.390631e-03
Zelda      -2.533472e-03
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 964 ms, sys: 102 ms, total: 1.07 s
Wall time: 1.01 s


name
Alice      -1.565881e-03
Bob        -3.429515e-03
Charlie    -1.754424e-03
Dan        -5.897803e-04
Edith       1.238215e-03
Frank       1.084501e-04
George     -1.650859e-03
Hannah     -4.881836e-04
Ingrid     -9.907006e-04
Jerry       2.702808e-04
Kevin      -1.816574e-03
Laura      -1.765015e-03
Michael    -1.616450e-03
Norbert     2.764356e-03
Oliver     -2.794524e-03
Patricia    2.561128e-03
Quinn       2.019727e-03
Ray         1.039835e-03
Sarah      -1.200362e-03
Tim        -1.493787e-04
Ursula      2.147744e-03
Victor      7.983312e-07
Wendy       4.596371e-03
Xavier      3.172719e-03
Yvonne     -2.390631e-03
Zelda      -2.533472e-03
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 961 ms, sys: 45.3 ms, total: 1.01 s
Wall time: 967 ms


name
Alice      -1.565881e-03
Bob        -3.429515e-03
Charlie    -1.754424e-03
Dan        -5.897803e-04
Edith       1.238215e-03
Frank       1.084501e-04
George     -1.650859e-03
Hannah     -4.881836e-04
Ingrid     -9.907006e-04
Jerry       2.702808e-04
Kevin      -1.816574e-03
Laura      -1.765015e-03
Michael    -1.616450e-03
Norbert     2.764356e-03
Oliver     -2.794524e-03
Patricia    2.561128e-03
Quinn       2.019727e-03
Ray         1.039835e-03
Sarah      -1.200362e-03
Tim        -1.493787e-04
Ursula      2.147744e-03
Victor      7.983312e-07
Wendy       4.596371e-03
Xavier      3.172719e-03
Yvonne     -2.390631e-03
Zelda      -2.533472e-03
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html